# Benchmarking time series regression models

Time series extrinsic regression, first properly defined in [1] then recently
extended in [2], involves predicting a continuous target variable based on a time
series. It differs from time series forecasting regression in that the target is
not formed from a sliding window, but is some external variable.

This notebook shows you how to use aeon to get benchmarking datasets with aeon and how
 to compare results on these datasets with those published in [2].

## Loading/Downloading data

aeon comes with two regression problems in the datasets module. You can load these
with single problem loaders or the more general load_regression function.

In [20]:
from aeon.datasets import load_cardano_sentiment, load_covid_3month, load_regression

trainX, trainy = load_covid_3month(split="train")
testX, testy = load_regression(split="test", name="Covid3Month")
X, y = load_cardano_sentiment()  # Combines train and test splits
print(trainX.shape, testX.shape, X.shape)

there are currently 63 problems in the TSER archive hosted on
timeseriesclassification.com. These are listed in the file datasets.tser_datasets

In [21]:
from aeon.datasets.tser_datasets import tser_soton

print(sorted(list(tser_soton)))

You can download these datasets directly with aeon load_regression function. By
default it will store the data in a directory called "local_data" in the datasets
module. Set ``extract_path`` to specify a different location.


In [22]:
small_problems = [
    "CardanoSentiment",
    "Covid3Month",
]

for problem in small_problems:
    X, y = load_regression(name=problem)
    print(problem, X.shape, y.shape)

This stores the data in a format like this

If you call the function again, it will load
from disk rather than downloading
again.
 You can specify train/test splits.

In [23]:
for problem in small_problems:
    trainX, trainy = load_regression(name=problem, split="train")
    print(problem, X.shape, y.shape)

## Evaluating a regressor on benchmark data

With the data, it is easy to assess an algorithm performance. We will use the
DummyRegressor as a baseline, and the default scoring



In [24]:
from sklearn.metrics import mean_squared_error

from aeon.regression import DummyRegressor

dummy = DummyRegressor()
performance = []
for problem in small_problems:
    trainX, trainy = load_regression(name=problem, split="train")
    dummy.fit(trainX, trainy)
    testX, testy = load_regression(name=problem, split="test")
    predictions = dummy.predict(testX)
    mse = mean_squared_error(testy, predictions)
    performance.append(mse)
    print(problem, " Dummy score = ", mse)

## Comparing to published results

How does the dummy compare to the published results in [2]? We can use the method
get_estimator_results to obtain published results.

In [25]:
from aeon.benchmarking import get_available_estimators, get_estimator_results

print(get_available_estimators(task="regression"))
results = get_estimator_results(
    estimators=["DrCIF", "FreshPRINCE"],
    task="regression",
    datasets=small_problems,
    measure="mse",
)
print(results)

this is organised as a dictionary of dictionaries. because we cannot be sure all
results are present for all datasets.

In [26]:
from aeon.benchmarking import get_estimator_results_as_array

results, names = get_estimator_results_as_array(
    estimators=["DrCIF", "FreshPRINCE"],
    task="regression",
    datasets=small_problems,
    measure="mse",
)
print(results)
print(names)

we just need to align our results from the website so they are aligned with the
results from our dummy regressor

In [27]:
import numpy as np

paired_sorted = sorted(zip(names, results))
names, _ = zip(*paired_sorted)
sorted_rows = [row for _, row in paired_sorted]
sorted_results = np.array(sorted_rows)
print(names)
print(sorted_results)

Do the same for our dummy regressor results

In [28]:
paired = sorted(zip(small_problems, performance))
small_problems, performance = zip(*paired)
print(small_problems)
print(performance)
all_results = np.column_stack((sorted_results, performance))
print(all_results)
regressors = ["DrCIF", "FreshPRINCE", "Dummy"]

## Comparing Regressors

aeon provides visualisation tools to compare regressors.

## Comparing two regressors

We can plot the results against each other. This also presents the wins and losses
and some summary statistics.

In [31]:
from aeon.visualisation import plot_pairwise_scatter

fig, ax = plot_pairwise_scatter(
    all_results[:, 1],
    all_results[:, 2],
    "FreshPRINCE",
    "Dummy",
    metric="mse",
    lower_better=True,
)


### Comparing multiple regressors

We can plot the results of multiple regressors on a critical difference diagram,
which shows the average rank and groups estimators by whether they are significantly
different from each other.

In [33]:
from aeon.visualisation import plot_critical_difference

res = plot_critical_difference(
    all_results,
    regressors,
    lower_better=True,
)

In [36]:
from aeon.visualisation import plot_boxplot_median

res = plot_boxplot_median(
    all_results,
    regressors,
)